In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def sector_data(sector_page_link):
    response = requests.get(sector_page_link)
    results_page = BeautifulSoup(response.content, 'lxml')
    
    sector_change = results_page.find('div', class_='tab-content-wrapper').find_all('span')[1].get_text()
    sector_market_cap = results_page.find('div', class_='tab-content-wrapper').find_all('span')[3].get_text()
    sector_market_weight = results_page.find('div', class_='tab-content-wrapper').find_all('span')[5].get_text()
    
    return sector_change, sector_market_cap, sector_market_weight

def sector_performance():
    stats_list = list()
    sector_list = list()
    measure_list = list()

    url = "https://eresearch.fidelity.com/eresearch/goto/markets_sectors/landing.jhtml"

    results_page = requests.get(url)
    soup = BeautifulSoup(results_page.content, 'lxml')
    all_performance = soup.find_all('a', class_='heading1')

    for sectors in all_performance:
        name = sectors.find('strong').get_text()
        link = "https://eresearch.fidelity.com" + sectors.get('href')
        change, mktcap, mktwgt = sector_data(link)
        stats_list.append((change, mktcap, mktwgt))
        sector_list.append(name)

    measure_list = 'Sector_Change', 'Market_Capital', 'Market_Weight'
    sector_table = pd.DataFrame(stats_list, index = sector_list, columns = measure_list)
    
    return sector_table

# Sector performance from Fidelity
sector_performance()

,Sector_Change,Market_Capital,Market_Weight
Communication Services,+0.07%,$4.46T,9.80%
Consumer Discretionary,-0.32%,$4.73T,9.73%
Consumer Staples,+0.20%,$3.41T,7.50%
Energy,+0.72%,$3.47T,5.51%
Financials,-0.72%,$7.16T,13.82%
Health Care,+0.60%,$5.66T,15.61%
Industrials,-0.03%,$3.89T,9.51%
Information Technology,-0.86%,$6.85T,19.70%
Materials,+0.58%,$1.76T,2.67%
Real Estate,+0.29%,$1.20T,2.95%


In [1]:
stock = input('Enter a stock ticker: ').upper()

Enter a stock ticker: jd


In [2]:
def get_sector():
    import requests
    from bs4 import BeautifulSoup
    url = 'https://finance.yahoo.com/quote/' + stock + '/profile?p=' + stock

    response = requests.get(url)
    results_page = BeautifulSoup(response.content, 'lxml')
    profile = results_page.find('div', class_='asset-profile-container')
    sector = profile.find_all('strong')[0].get_text()
    industry = profile.find_all('strong')[1].get_text()

    return sector, industry
# sector and industry from Yahoo Finance
get_sector()

('Consumer Cyclical', 'Specialty Retail')

In [3]:
def get_competitors(stock):
    import requests
    from bs4 import BeautifulSoup
    url = 'https://www.nasdaq.com/symbol/' + stock + '/stock-comparison'

    response = requests.get(url)
    results_page = BeautifulSoup(response.content, 'lxml')
    table = results_page.find('div', class_='genTable thin')
    competitors = list()
    for i in range(4):
        competitors.append(table.find_all('input')[i].get('value'))
    
    return competitors
# competitors from nasdaq
get_competitors(stock)

['ORLY', 'AZO', 'ULTA', 'IAC']

In [13]:
def get_financial_stats(ticker):
    import requests
    from bs4 import BeautifulSoup
    url = "https://finance.yahoo.com/quote/" + ticker + "/key-statistics?p=" + ticker
    
    response = requests.get(url)
    results_page = BeautifulSoup(response.content,'lxml')
    stats_lists = results_page.find_all('table')
    test_list = list()
    for i in stats_lists:
        stat_name = i.find_all('span')
        stock_stat = i.find_all('td',class_='Fz(s) Fw(500) Ta(end)')
        for j in range(len(stock_stat)):
            if not stat_name[j].get_text() == 'N/A':
                test_list.append((stat_name[j].get_text(),stock_stat[j].get_text()))
    return test_list

In [65]:
def get_row(ticker,measure_list):
    from pandas_datareader import data as web
    a = get_financial_stats(ticker)
    row = list()
    row.append(web.DataReader(ticker,"yahoo").iloc[-1,3])
    for i in measure_list:
        if not i == "Price":
            appended = 0
            for j in a:
                if j[0] == i:
                    row.append(j[1])
                    appended = 1
            if appended == 0:
                row.append('N/A')   
    return row

In [19]:
def comp_stat_table(stock):
    import pandas as pd
    measure_list = ['Price','Market Cap (intraday)','Diluted EPS','Book Value Per Share','Quarterly Revenue Growth','Quarterly Earnings Growth','Trailing Annual Dividend Yield','5 Year Average Dividend Yield','Payout Ratio','Trailing P/E','PEG Ratio (5 yr expected)','Beta (3Y Monthly)']
    r = get_competitors(stock)
    r.insert(0, stock)
    
    s_row = get_row(stock,measure_list)
    a_row = get_row(r[1],measure_list)
    b_row = get_row(r[2],measure_list)
    c_row = get_row(r[3],measure_list)
    d_row = get_row(r[4],measure_list)

    stats_list = [s_row,a_row,b_row,c_row,d_row]
    final_table = pd.DataFrame(stats_list,index = r,columns = measure_list)
    return final_table
comp_stat_table(stock)

,Price,Market Cap (intraday),Diluted EPS,Book Value Per Share,Quarterly Revenue Growth,Quarterly Earnings Growth,Trailing Annual Dividend Yield,5 Year Average Dividend Yield,Payout Ratio,Trailing P/E,PEG Ratio (5 yr expected),Beta (3Y Monthly)
JD,21.230000,30.72B,-0.06,6.12,31.20%,N/A,N/A,N/A,N/A,N/A,0.49,1.38
ORLY,346.779999,27.78B,15.89,6.20,6.10%,29.00%,N/A,1/2,N/A,21.83,1.29,1.21
AZO,809.070007,20.68B,48.77,N/A,1.30%,-7.70%,N/A,1/2,N/A,16.59,1.32,1.06
ULTA,297.790009,17.8B,10.25,N/A,15.40%,29.90%,"May 15, 2012",N/A,N/A,29.06,1.35,0.85
IAC,177.960007,14.86B,4.96,31.15,33.30%,-18.90%,0.00%,"Dec 1, 2015",2/1,35.86,1.82,1.58


In [69]:
import datetime
import dateutil.relativedelta
import numpy as np
import pandas_datareader.data as web

stocks = stock, get_competitors(stock)
five_years_ago = (datetime.date.today() - dateutil.relativedelta.relativedelta(years=5)).strftime('%Y-%m-%d')
today = datetime.date.today().strftime('%Y-%m-%d')


# df = web.DataReader(np.hstack(stocks),'yahoo',five_years_ago,today)['Adj Close']
df = web.get_data_yahoo(np.hstack(stocks),five_years_ago,today)['Adj Close']
df

Symbols,AZO,IAC,JD,ORLY,ULTA
Date,,,,,
2013-12-02,459.619995,54.997612,NaN,124.570000,126.930000
2013-12-03,460.510010,54.939850,NaN,124.360001,125.010002
2013-12-04,456.019989,54.670250,NaN,121.080002,123.480003
2013-12-05,460.000000,53.977001,NaN,120.959999,118.000000
2013-12-06,459.600006,54.169567,NaN,125.000000,93.760002
2013-12-09,457.339996,55.498295,NaN,123.860001,90.209999
2013-12-10,471.859985,57.106239,NaN,125.019997,94.389999
2013-12-11,467.790009,56.692219,NaN,124.860001,93.209999
2013-12-12,465.899994,56.268566,NaN,124.919998,91.239998


In [10]:
# def get_stats(ticker):
#     url = 'https://finance.yahoo.com/quote/' + ticker + '/history?p=' + ticker

#     response = requests.get(url)
#     results_page = BeautifulSoup(response.content,'lxml')
#     price = results_page.find('div').find_all('td')[5].get_text()

#     url = 'https://finance.yahoo.com/quote/' + ticker + '/key-statistics?p=' + ticker

#     response = requests.get(url)
#     results_page = BeautifulSoup(response.content,'lxml')
#     stats = results_page.find_all('table')
#     test_list = list()
#     for i in stats:
#         stat_name = i.find_all('span')
#         stock_stat = i.find_all('td', class_='Fz(s) Fw(500) Ta(end)')
#         for j in range(len(stat_name)):
#             if not stat_name[j].get_text() == 'N/A':
#                 test_list.append((stat_name[j].get_text(), stock_stat[j].get_text()))
#     test_list.insert(0, ('Price', price))
#     return test_list
# get_stats(stock)

[('Price', '179.55'),
 ('Market Cap (intraday)', '852.04B'),
 ('Enterprise Value', '865.77B'),
 ('Trailing P/E', '15.08'),
 ('Forward P/E', '12.11'),
 ('PEG Ratio (5 yr expected)', '1.03'),
 ('Price/Sales', '3.21'),
 ('Price/Book', '7.97'),
 ('Enterprise Value/Revenue', '3.26'),
 ('Enterprise Value/EBITDA', '10.58'),
 ('Fiscal Year Ends', 'Sep 29, 2018'),
 ('Most Recent Quarter', 'Sep 29, 2018'),
 ('Profit Margin', '22.41%'),
 ('Operating Margin', '26.69%'),
 ('Return on Assets', '11.96%'),
 ('Return on Equity', '49.36%'),
 ('Revenue', '265.59B'),
 ('Revenue Per Share', '53.60'),
 ('Quarterly Revenue Growth', '19.60%'),
 ('Gross Profit', '101.84B'),
 ('EBITDA', '81.8B'),
 ('Net Income Avi to Common', '59.53B'),
 ('Diluted EPS', '11.91'),
 ('Quarterly Earnings Growth', '31.80%'),
 ('Total Cash', '66.3B'),
 ('Total Cash Per Share', '13.97'),
 ('Total Debt', '114.48B'),
 ('Total Debt/Equity', '106.85'),
 ('Current Ratio', '1.12'),
 ('Book Value Per Share', '22.53'),
 ('Operating Cash Flow